In [1]:
import tensorflow as tf
from keras.datasets import cifar10
from  keras.utils import np_utils


# loading the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# # building the input vector from the 32x32 pixels
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)


Shape before one-hot encoding:  (50000, 1)
Shape after one-hot encoding:  (50000, 10)


In [2]:
print(Y_test[1])

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu',),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()
# model.save('cifar.keras')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                11530     
Total params: 21,674
Trainable params: 21,674
Non-trainable params: 0
__________________________________________________

In [5]:
history = model.fit(X_train, Y_train, batch_size=64, epochs=2, validation_data=(X_test, Y_test))


Epoch 1/2
782/782 [==============================] - 22s 28ms/step - loss: 1.5783 - accuracy: 0.4393 - val_loss: 1.3452 - val_accuracy: 0.5269
Epoch 2/2
782/782 [==============================] - 21s 27ms/step - loss: 1.2596 - accuracy: 0.5628 - val_loss: 1.2060 - val_accuracy: 0.5797


In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X_train.astype('float32')).batch(1).take(100):
      yield [input_value]   
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce full-int8 quantization (except inputs/outputs which are always float)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_data_gen
model_tflite = converter.convert()

# tflite_model = converter.convert()
model_length = open("model.tflite", "wb").write(model_tflite)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /var/folders/mv/1txqzr191f1b8gyq0sjcwyth0000gn/T/tmppxzgjdn4/assets


In [7]:
!xxd -i 'model.tflite' > 'model.cc'

In [8]:
#  Read in the file
with open('model.cc', 'r') as file :
  filedata = file.read()

#include "model.h"
# Replace the target string
filedata = filedata.replace('unsigned char', 'alignas(16) const unsigned char' )
filedata = filedata.replace('unsigned int', 'const int' )
filedata = '#include "model.h" \n \n' \
    + "// Keep model aligned to 8 bytes to guarantee aligned 64-bit accesses. \n\n" \
    + filedata
# Write the file out again
with open('model.cc', 'w') as file:
  file.write(filedata)

In [9]:
read_h_file = open('model.h', 'r') 
old_lines = read_h_file.readlines()
new_lines = []    
for line in old_lines:
    if "model_len" in line:
        new_lines.append("const int model_len = {};".format(model_length) )
    else:
        new_lines.append(line)
# print (new_lines)
read_h_file.close()

write_header_file = open('model.h', 'w') 
write_header_file.writelines(new_lines)
write_header_file.close()